# Healthy Streets of Los Angeles Mobility plan analysis

This project finds out how well LA Mobility Plan 2035 was implemented to this day (November 2023) for protected and unprotected bike lanes and protected bus lanes. Also it finds the same data for Neighborhood Enhanced Network bike lanes.

Sources:
* Current Bike Lanes: https://geohub.lacity.org/datasets/ladot::city-of-los-angeles-bikeways-1/explore
* Mobility Plan bike lanes: https://geohub.lacity.org/datasets/bicycle-enhanced-network/explore 
* Mobility Plan bike paths: https://geohub.lacity.org/datasets/lahub::green-network-bicycle-paths-1/explore </br >
Mobility Plan bike lanes tiers are in "Bicycle_N" column
* Neighborhood Enhanced Network bike lanes https://geohub.lacity.org/datasets/lahub::neighborhood-enhanced-network/explore
* Current Bus Lanes: https://www.google.com/maps/d/u/1/edit?mid=10co-9X_jGrJtGBA9qVhwWOqUpQNng2dx&usp=sharing 
* Mobility plan Bus lanes: https://geohub.lacity.org/datasets/transit-enhanced-network/explore?location=34.018131%2C-118.376481%2C11.68

Compare: 
* Actual Bus Lanes = Mobility Plan tier 3
* Actual Bike Lanes Class 4 = Protected (Mobility Plan tier 1)
* Actual Bike Lanes Class 2 = Unprotected (Mobility Plan tier 2 and 3)
* Actual Bike Lanes Class 1 = Bicycle Enhanced Network
* Actual Bike Lanes Class 3 = Neighborhood Enhanced Network

Deliverable: 
* What % of the mobility plan has been implemented
* Geojson result
* year > 2015, because Mobility Plan was approved in late 2015

Any questions - elena.sunchugasheva@gmail.com

In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import folium
import datetime

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)

In [9]:
today = datetime.datetime.now().strftime("%Y_%m_%d")
print(today)

2023_12_10


## functions

In [10]:
def show_map(gdf1, name1, color1, gdf2, name2, color2):
    
    print(
            f'{color1}: {name1}, {color2}: {name2}'
        )
    
    f = folium.Figure(width=1000, height=500)
    
    m = gdf1.explore(
        name = name1,
        color = color1
    ).add_to(f)

    map_2 = gdf2.explore(
        m=m,  # pass the map object
        name = name2,
        color = color2
    )

    folium.TileLayer(
        'CartoDB positron',
        show=False
    ).add_to(m) 

    folium.LayerControl().add_to(m)
    
    return m

In [3]:
def get_xy(line):
    if line.has_z:
        xy_line = [xy[:2] for xy in list(line.coords)]
    return LineString(xy_line)

In [4]:
def buffer(gdf, radius, proj='EPSG:4326', proj_calc='EPSG:3857'):
    '''
    convert a gdf of linestrings into a gdf of polygons with radius
    - gdf - GeoDataFrame, has column "geometry"
    - radius - radius of bufferm meters
    - proj - projection of the original dataset
    - proj_calc='EPSG:3857' - projection for calculations
    '''  
    gdf = gdf.copy()
    gdf_calc = gdf.to_crs(proj_calc)
    #print('data proj:', proj, '\ncalculation proj: ', proj_calc)
    gdf['buffered'] = gdf_calc.buffer(radius, cap_style=2).to_crs(proj)
    gdf.set_geometry('buffered', inplace=True)
    
    # merge all intersecting buffered polygons
    gdf_return = gpd.GeoDataFrame(
        geometry=[gdf.unary_union]
    ).explode(
        index_parts=False
    ).reset_index(
        drop=True
    )
    gdf_return.geometry.crs = proj
    
    return gdf_return

In [78]:
def get_lengths(gdf, column='geometry', proj='EPSG:4326', proj_calc='EPSG:3857'):
    '''
    return a gdf with a column of linestring length in proj_calc units (m)
    - gdf - GeoDataFrame
    - column - column to be used for length calculation if not "geometry"
    - proj - projection of the original dataset
    - proj_calc='EPSG:3857' - projection for calculations
    '''
    if column:
        gdf.set_geometry(column, inplace=True)
    # convert projection to proj_calc, if default - the units will be in meters
    gdf_m = gdf.to_crs(proj_calc)
    gdf['length_m'] = gdf_m.length
    
    return gdf

In [100]:
def compare_length(
        name,
        gdf_plan,
        gdf_actual,
        conditions,
        radius = 10,
        year = None,
        column_year = 'Year_',
        print_map = True,
        test_map = False
    ):
    '''
    - name 
    - gdf_plan - GeoDataFrame of Mobility plan
    - gdf_actual - GeoDataFrame of existing roads, has a column_year
    - conditions - filter specific types of bike/bus lanes
    - radius - radius in meters to widen the line of actual roads
    - year - filter data after construction year
    - column_year - column of construction year
    - print_map - show final map
    - test_map - show interim map
    '''
    print(f'{name} for year after {year}:')
    gdf_plan = gdf_plan.copy()
    gdf_actual = gdf_actual.copy()
    
    if year:
        gdf_actual = gdf_actual[gdf_actual[column_year]>year].copy()
    if conditions['plan']:
        if 'bike' in key:
            gdf_plan = gdf_plan[gdf_plan.BICYCLE_N.isin(
                conditions['plan']
            )].copy()
        if 'bus' in key:
            gdf_plan = gdf_plan[gdf_plan.TRANSIT_N.isin(
                conditions['plan']
            )].copy()
    if conditions['actual']:
            gdf_actual = gdf_actual[gdf_actual.Class.isin(
                conditions['actual']
            )].copy()
    
    print(f'radius = {radius} m')
    # widen linestring to polygon
    gdf_actual_buffer = buffer(
        gdf_actual, 
        radius = radius,
        proj = gdf_actual.geometry.crs
    )
    if test_map:
        display(
            show_map(
                gdf2 = gdf_actual_buffer[['geometry']],
                name2 = 'implemented', 
                color2 = 'blue',
                gdf1 = gdf_plan[['OBJECTID', 'geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )
    
    # intersect polygons of actual and planned paths
    gdf_implemented = gpd.overlay(
        gdf_plan, 
        gdf_actual_buffer, 
        how='intersection',
        keep_geom_type=False
    )
    gdf_implemented = get_lengths(gdf_implemented)
    gdf_implemented = gdf_implemented[
        round(gdf_implemented.length_m/(radius*2), 2)>1.01
    ].reset_index(drop=True)
    
    # map of buffered planned (green) and implemented (red) paths
    if print_map:
        display(
            show_map(
                gdf2 = gdf_implemented[['OBJECTID', 'geometry']],
                name2 = 'implemented', 
                color2 = 'red',
                gdf1 = gdf_plan[['OBJECTID', 'geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )
        
    print(
        'since', year, 'year:',
        round(get_lengths(gdf_implemented).length_m.sum()/1609.34, 2), 'miles implemented',
        round(get_lengths(gdf_plan).length_m.sum()/1609.34, 2), 'miles planned',
        '\nimplemented lanes records:', gdf_implemented.shape[0],
        '\nplanned lanes records:', gdf_plan.shape[0]        
    )
    
    percentage = round(
        get_lengths(gdf_implemented).length_m.sum()/
        get_lengths(gdf_plan).length_m.sum()*100, 
        2
    )

    return percentage, gdf_implemented

## bike lanes

In [ ]:
bike_plan_file = open('Bicycle_Enhanced_Network.geojson')
bike_plan_geo_raw = gpd.read_file(bike_plan_file)
bike_actual_file = open('City_of_Los_Angeles_Bikeways.geojson')
bike_actual_geo_raw = gpd.read_file(bike_actual_file)

In [ ]:
bike_plan_geo = bike_plan_geo_raw.copy()
bike_actual_geo = bike_actual_geo_raw.copy()
print(
    'Mobility plan records:',
    bike_plan_geo.shape[0],
    '\nActual bike lanes records:',
    bike_actual_geo.shape[0],
    '\nActual lanes without construction year:',
    bike_actual_geo[bike_actual_geo.Year_.isnull()].shape[0]
)
display(bike_plan_geo.head(1))
display(bike_actual_geo.tail(1))

### Clean up data/figure out intersections

In [ ]:
# actual bike paths surprisingly have z coordinate,
# so following is the best way to get rid of it (all z==0) I came up with

bike_actual_geo = bike_actual_geo.explode(ignore_index=False, index_parts=False)
bike_actual_geo.geometry = bike_actual_geo.geometry.apply(lambda x: get_xy(x))
display(bike_actual_geo.head(1))

<b>Geo intersecting data</b></br>
* two linestrings</br>
As we see below, lines for planned and actual bike paths are not exactly the same (but actually close, see the map below).
* a linesting and polygon</br>
We can artificially widen one of the linestrings into a polygon so it would overlap with the other linestring - see function 'buffer'.

We chose converting to polygon actual data over planned for following reasons:
1. we won't double count bike paths, planned data has one line where actual data sometines two (one bike lane one way and one lane the other way).
2. we're comparing result (implemented bike lanes) to planned data, so there we won't depend on possible irregularities of existing bike paths</br>

In [ ]:
# show_map(
#     gdf1 = bike_actual_geo[['SECT_ID', 'geometry']],
#     name1 = 'actual', 
#     color1 = 'blue',
#     gdf2 = bike_plan_geo[['OBJECTID', 'geometry']],
#     name2 = 'plan', 
#     color2 = 'green',
# )

### calculate percentages for protected/unprotected lanes

In [33]:
bike_conditions = {
    'protected bike lane': {'plan': [1], 'actual': [4]},
    'unprotected bike lane': {'plan': [2, 3], 'actual': [2]}
}
results = []
result_gdfs = []

for key in bike_conditions.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            conditions = bike_conditions[key],
            gdf_actual = bike_actual_geo[[
                            'OBJECTID_12',
                            'SECT_ID',
                            'Class',
                            'Year_',
                            'geometry'
                        ]],
            gdf_plan = bike_plan_geo[[
                            'OBJECTID',
                            'BICYCLE_N',
                            'geometry'
                       ]],
            print_map = False,
            test_map = False,
            year = year,
            column_year = 'Year_'
        )
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
        
        print(
            percentage,
            f'% of {key} implemented after {year}',
            '\n'
        )
        
    print('\n----')    

### Bike Paths vs class 1

In [ ]:
bike_plan_paths = open('Bicycle_Enhanced_Network_Paths.geojson')
bike_paths_geo_raw = gpd.read_file(bike_plan_paths)

In [ ]:
bike_path_geo = bike_paths_geo_raw.copy()
print(bike_path_geo.shape[0])
display(bike_path_geo.head())

Geometry for the city's bike path file is too inaccurate to run a spatial comparison i.e. it misses the majority of Ballona Creek </br>
Fortunately there are no bike paths completed that aren't on the mobility plan, so we can use the city's bikeway map of Class 1 routes and sort by year and just compare total lengths</br>
<span style="color:red">(this done in the next couple of cells, but possible solution is to widen the buffer area - parameter radius, see below)</span>

In [ ]:
# # return length of all planned path
# planned_lengths = get_lengths(bike_path_geo)
# m_planned = planned_lengths.length_m.sum()
# 
# # return length of all class 1 routes
# actual_paths = bike_actual_geo.loc[bike_actual_geo['Class'] == 1].copy()
# path_built = get_lengths(actual_paths)
# m_built = path_built.length_m.sum()
# 
# # return length of all class 1 routes completed after 2015
# actual_paths_2015 = actual_paths.loc[actual_paths['Year_'] > 2015].copy()
# path_built_2015 = get_lengths(actual_paths_2015)
# m_built_2015 = path_built_2015.length_m.sum()
# 
# print(round(m_built/m_planned * 100, 2), '% of bike paths implemented total ')
# print(round(m_built_2015/m_planned * 100, 2), '% of bike paths implemented after 2015 ')

^ output </br>
37.37 % of bike paths implemented total </br>
2.74 % of bike paths implemented after 2015 </br>

In [ ]:
# bike_path_conditions = {
#     'bike path': {'plan': None, 'actual': [1]},
# }

# for key in bike_path_conditions.keys():
#     for year in (None, 2015):
#         percentage, result_gdf = compare_length(
#             name = key,
#             conditions = bike_path_conditions[key],
#             gdf_actual = bike_actual_geo[[
#                             'OBJECTID_12',
#                             'SECT_ID',
#                             'Class',
#                             'Year_',
#                             'geometry'
#                         ]],
#             gdf_plan = bike_path_geo,
#             print_map = True,
#             year = year,
#             column_year = 'Year_',
#             test_map = False  
#         )
        
#         results.append([key, year, percentage])
#         result_gdfs.append([key, year, result_gdf])

#         print({key})
        
#         print(
#             percentage,
#             f'% of {key} implemented after {year}',
#             '\n'
#         )
        
#     print('\n----')    

^ output:</br>
bike path</br>
since None year: </br>
implemented lanes records: 45 </br>
planned lanes records: 146</br>
{'bike path'}</br>
11.55 % of bike path implemented after None </br>
</br>
bike path</br>
since 2015 year: </br>
implemented lanes records: 3 </br>
planned lanes records: 146</br>
{'bike path'}</br>
0.61 % of bike path implemented after 2015 </br>

In [ ]:
bike_class1_conditions = {
    'class1 bike lane': {'plan': None, 'actual': [1]},
}

for key in bike_class1_conditions.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            gdf_plan = bike_path_geo[['OBJECTID', 'geometry']],
            gdf_actual = bike_actual_geo[[
                'OBJECTID_12', 'Year_', 'Class','geometry'
            ]],
            conditions = {'plan': None, 'actual': [1]},
            radius = 120,
            print_map = True,
            test_map = True,
            year = year,
            column_year = 'Year_'
                )
        print(f'{percentage}% of bike path implemented after {year}')
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
        print('_______')

### NEN vs class 3

In [ ]:
bike_nen_file = open('Neighborhood_Enhanced_Network.geojson')
bike_nen_geo_raw = gpd.read_file(bike_nen_file)

In [ ]:
bike_nen_geo = bike_nen_geo_raw.copy()
print(bike_nen_geo.shape[0])
display(bike_nen_geo.head())

In [ ]:
bike_nen_conditions = {
    'NEN bike lane': {'plan': None, 'actual': [3]},
}

for key in bike_nen_conditions.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            conditions = bike_nen_conditions[key],
            gdf_actual = bike_actual_geo[[
                            'OBJECTID_12',
                            'SECT_ID',
                            'Class',
                            'Year_',
                            'geometry'
                        ]],
            gdf_plan = bike_nen_geo,
            print_map = False,
            year = year,
            column_year = 'Year_',
            test_map = False
        )
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
        
        print(
            percentage,
            f'% of {key} implemented after {year}',
            '\n'
        )
        
    print('\n----')    

## bus lanes

In [71]:
bus_plan_file = open('Transit_Enhanced_Network.geojson')
bus_plan_geo_raw = gpd.read_file(bus_plan_file)
bus_actual_file = open('bus_only_lanes_sat.geojson')
bus_actual_geo_raw = gpd.read_file(bus_actual_file)

In [72]:
bus_plan_geo = bus_plan_geo_raw.copy()
bus_actual_geo = bus_actual_geo_raw.copy()
print(
    'Mobility plan records:',
    bus_plan_geo.shape[0],
    '\nActual bus lanes records:',
    bus_actual_geo.shape[0]
)
display(bus_plan_geo.head())
display(bus_actual_geo.head())

Mobility plan records: 4462 
Actual bus lanes records: 22


,OBJECTID,CF,CASE_NUM,SOURCE,ADOPTDATE,CPA_1,CPA_2,TRANSIT_N,Shape__Length,geometry
0,1,15-0719,CPC-2013-910-GPA-SP-CA-MSC,,2015-08-11 00:00:00+00:00,WCH,None,1,428.828885,"LINESTRING (-118.37112 33.94526, -118.37497 33..."
1,2,15-0719,CPC-2013-910-GPA-SP-CA-MSC-M2,,2016-09-09 00:00:00+00:00,SLA,None,3,132.119712,"LINESTRING (-118.29166 33.96879, -118.29166 33..."
2,3,15-0719,CPC-2013-910-GPA-SP-CA-MSC-M2,,2016-09-09 00:00:00+00:00,WCH,None,1,358.473571,"LINESTRING (-118.37527 33.94525, -118.37765 33..."
3,4,15-0719,CPC-2013-910-GPA-SP-CA-MSC,,2015-08-11 00:00:00+00:00,WCH,None,1,33.032638,"LINESTRING (-118.37497 33.94525, -118.37503 33..."
4,5,15-0719,CPC-2013-910-GPA-SP-CA-MSC,,2015-08-11 00:00:00+00:00,SVY,None,2,78.145866,"LINESTRING (-118.38780 34.23308, -118.38780 34..."


,Name,description,timestamp,begin,end,altitudeMode,tessellate,extrude,visibility,drawOrder,icon,lanes,Installed,Year_,Hours,geometry
0,Spring,lanes: 1<br>Installed: y<br>Year : 1974<br>24/...,None,None,None,None,1,0,-1,None,None,1.0,y,1974,1,"LINESTRING Z (-118.23948 34.05827 0.00000, -11..."
1,6th,lanes: 1<br>Installed: y<br>Year : 2020<br>24/...,None,None,None,None,1,0,-1,None,None,1.0,y,2020,0,"LINESTRING Z (-118.25754 34.05020 0.00000, -11..."
2,5th,lanes: 1<br>Installed: y<br>Year : 2020<br>24/...,None,None,None,None,1,0,-1,None,None,1.0,y,2020,0,"LINESTRING Z (-118.25602 34.05154 0.00000, -11..."
3,Sunset,lanes: 1<br>Installed: y<br>Year : 2013<br>24/...,None,None,None,None,1,0,-1,None,None,1.0,y,2013,0,"LINESTRING Z (-118.24626 34.06270 0.00000, -11..."
4,Flower,lanes: 1<br>Installed: y<br>Year : 2019<br>24/...,None,None,None,None,1,0,-1,None,None,1.0,y,2019,0,"LINESTRING Z (-118.25880 34.04869 0.00000, -11..."


In [73]:
# take a look at planned vs existing protected lanes
protected_bus_plan = bus_plan_geo[bus_plan_geo.TRANSIT_N >= 2].copy()
display(
            show_map(
                gdf2 = bus_actual_geo[['Name', 'geometry']],
                name2 = 'actual', 
                color2 = 'blue',
                gdf1 = protected_bus_plan[['OBJECTID', 'geometry']],
                name1 = 'plan', 
                color1 = 'green',
            )
        )

green: plan, blue: actual


In [101]:
bus_conditions = {
    'bus lane': {'plan': [3, 2], 'actual': None},
}

for key in bus_conditions.keys():
    for year in (None, 2015):
        percentage, result_gdf = compare_length(
            name = key,
            conditions = bus_conditions[key],
            radius = 20,
            gdf_actual = bus_actual_geo[[
                            'Name',
                            'geometry',
                            'Year_',
                            'Hours'
                        ]].copy(),
            gdf_plan = bus_plan_geo,
            print_map = False,
            test_map = False,
            year = year,
            column_year ='Year_' 
        )
        results.append([key, year, percentage])
        result_gdfs.append([key, year, result_gdf])
    
        print(
            percentage,
            f'% of {key} implemented after {year}',
            '\n'
        )
        
        print('\n----')    

bus lane for year after None:
radius = 20 m
since None year: 23 miles implemented 273 miles planned 
implemented lanes records: 257 
planned lanes records: 3064
8.53 % of bus lane implemented after None 


----
bus lane for year after 2015:
radius = 20 m
since 2015 year: 13 miles implemented 273 miles planned 
implemented lanes records: 135 
planned lanes records: 3064
4.67 % of bus lane implemented after 2015 


----


## save results

In [19]:
results_df = pd.DataFrame(
    columns=['transport type', 'year', 'implemented_perc'],
    data = results
)
display(results_df)

NameError: name 'results' is not defined

In [ ]:
results_df.to_csv(f'mobility_plan_percentage_implemented_{today}.csv', index=False)

In [ ]:
for df in result_gdfs:
    name = df[0].replace(' ', '_') + '_implemented'

    year = df[1]
    if year:
        name = name + f'_since_{year}'
    print(name)
    df[2].to_file(f'{name}_{today}.geojson', driver='GeoJSON')